# MSc. AI
## Capstone Project
### Darragh Minogue

### 1. Background


This project adopts the facility location problem to the school network in Ethiopia to expand access to secondary education. Of the 40,776 schools in the country, 37,039 offer primary and middle school education (grades 1-8), but only 3,737 offer secondary (grades 10-12). On average, middle schools are located more than 6km away from the nearest secondary school, meaning children face long journeys or they don't access secondary at all.

The expansion of secondary education is a major priority for the Ministry of Education, Ethiopia. Ambitious five year targets are set in the Education Sector Development Plans (ESDP). In 2013/2014, the Gross Enrollment Rate (GER) for lower secondary was 38.9% (3,466,972/8,914,837 children) and a target of 74% (6,596,979/8,914,837 children) was set. However, by 2019/2020, the actual GER was 51.05% (4,551,024/8,914,837 children), a more modest increase of 12.15% rather than 35.1%.  In the latest five year ESDP (2020/2021-2024/2025), an increase of 24% was set, but under the key-performance indicator of 'Number of newly established secondary schools', it says TBD- for 'To Be Determined'. Therein lies part of the problem: planning. Often, a portion of the budget is allocated for secondary, then each region decides how and where to spend it. Planners then, at the macro level, identify primary schools with the highest demand (i.e. largest enrollment) and seek to construct schools to serve that particular school population. While this can often yeild good results, the goal of this project is to improve on this approach in a way that allows for the potential construction of secondary schools across a catchment of multiple primary schools, if enrollment is sufficient and distance is minimal. Meta-heuristic optimisation techniques are used to achieve this.

The project makes use of readily available data from the Ministry of Education, Ethiopia, mainly 1) enrollment data from the Education Managament Information System (EMIS) and 2) geolocation data, recently obtained from the World Bank in 2020. Since national population and census data is inaccurate and outdated in Ethiopia, this was not considered a reliable source for this project. Instead, it is assumed that middle schools are located within the community and new secondary schools should therefore be constructed close to existing middle schools.

### 2. The Problem

In the problem, there is a list of existing middle schools and secondary schools in Ethiopia, their enrollment and location data (region, zone, district, longitude and latitude). The aim is to expand the secondary school network by constructing new secondary schools that best serve the demand from the existing middle school network. No limit is placed on capacity as this is determined by budget which is not known in advance and subject to change. Key to solving the problem is understanding that when a school is located within a village, there is no impact of distance on enrolment nor dropout. However, beyond the aspirational norm of 1-2km, distance affects initial access to school but also creates barriers to retention, completion and transition to higher level. As such, the aim is to minimise distance, but construct schools in locations with the highest demand.

### 3. Approach

<b> Glossary of terms</b>
* MS: Middle Schools
* SS: Secondary Schools
* EE: Expected Enrollment
* EEI: Expected Enrollment Increase
* Feeder: a middle school that is part of the secondary school catchment area.
* x: proposed new secondary schools (the genotype).
* d: distance in km between primary schools
* catchment: a middle school is considered within a catchment if it's 
* n: total number of proposed new SS to construct

This solution proposes new SS locations that can provide the highest estimated enrollment from feeder MS whilst also ensuring they are constructed within at minimal distance. Given the multitude of different languages and the decentralised Government of Ethiopia, models are developed at a regional level.

Two different algorithms are used in this project to find an optimal solution: Random Search, and CMA-ES. In each of the algorithms, the objective function f() aims to <b> maximise overall expected enrollment </b> given a set of feasible locations: x. Feasibility of the locations or the genotype, x, is controlled using a box boundary of longitudes and latitudes for a given region. Initial starting points are also provided within the box boundary using a generate_random_sp() helper function. 

The fitness is determined using an expected_enrollment function. This takes in the parameters: 1) the location of existing MS, 2) the enrollment of existing MS, 3) x, and 4) the current distance between existing MS and existing SS. The function then calculates the haversine distance in kilometers (km) between each new SS in x and all MS within a given region. This vector, <b> d </b>, is of size: (proposed_schools, len(ps)). Using this data, the algorithm then addresses three key challenges:

1) **The need to identify the MS with the minimal distance to the new SS**. In this case, if a MS is close to two or more new SS, only the closest is selected. But if the MS is closer to an existing SS, then it's ignored as it's already a feeder school to the existing SS.

2) **The need to estimate the expected enrollment of the new SS, based on the distance to its feeder MS**. If a school is beyond a certain threshold distance in km, then the MS should not be considered a feeder school for a SS. To handle this, situation, a helper function is used to estimate the expected enrollment from a feeder MS to the nearby SS. It takes in as parameters a) distance to nearest SS and b) MS enrollment. It then makes some assumptions about distance to return the expected enrollment per school. Theunynck (2014) recommends the adoptation of a norm of 2 or 3km for junior secondary schools and the function therefore assumes that if a school is constructed within 3km of a MS, there is no negative effect on expected enrollment of it's nearby SS. Above 3km, a linear dropoff is assumed between 3-5km. If a MS is located more than 5km away from the new location proposed, it is expected that zero students will attend from that feeder school.

3) **The need to divert expected enrollment from a MS if it is currently a feeder school to an existing SS but the new SS proposed is closer**. This is dealt with by subtracting the expected enrollment from the MS to the existing SS using the shape function, and then allocating it to the new SS which is closer. 

A fitness evaluation budget of 10,000 is set to ensure there are sufficient iterations to achieve convergance. Each algorithm is run 30 times with different random starting points. The results are stored in a csv file, with the top 4 results plotted below. 

### Assumptions and Limitations

1. It is assumed that 1km is equivalent to a 15 minute walk for children (Theunynck, S. 2014: p6).
2. Distance is calculated using the haversine function and as a result distance is calculated as a straight line. The travel distances could therefore be further. Ethiopia is not well mapped and since most children are walking to school using other means like Google Maps API don't yeild useful results on a large scale and don't factor in more informal walking routes. Final results require close inspection for elevation and other issues that might impact walking distance or construction e.g. buildings, rivers. The final results should therefore be observed as an approximation and using a tool like ArcGIS or QGIS, the the results are observed for these types of obstacles. 
3. It is assumed that children beyond 5km are not likely to attend, but in some cases, this is not true. Some children walk extremely long distances to attend secondary school, while others stay with relatives or family friends to attend a school that is beyond 5km. Despite this being a reality, this shouldn't guide the construction as the goal is to minimise distance and create more equitable access to secondary. 

In [380]:
# Key imports
import pandas as pd
import numpy as np
from haversine import haversine, haversine_vector, Unit # for distance
import geopandas as gpd
import matplotlib.pyplot as plt
import cma
import time
import os.path

# Supress the scientific notation on numpy for easierx reading.
np.set_printoptions(suppress=True)

In [381]:
df = pd.read_csv('data/clean_dataset_final.csv', converters={'point': pd.eval}) # read in cleaned dataset.
gdf_region = gpd.read_file('eth_shape_files/json//eth_admin1v2.json') # read in geojson

In [382]:
# drop Afar as only 5% of schools are mapped.
df = df[df['ADM1_EN'] != 'Afar']

# Helper Functions

In [383]:
def shape(distance, enrollment):
    """ Returns the expected enrollment based on distance.
    
        If distance is less than 3km, all children are expected to contribute to the supply of a new school. 
        The full enrollment is returned. If above 5km, the school is considered too far and no children contribute to 
        the supply. Zero enrollment is returned. If between 3-5km, a linear dropoff is assumed. e.g. for 100 children
        at a distance of 4km, 50 is returned. 

        Parameters:
            distance (float): the distance in km of a school to the next level school.
            enrollment (int): the enrollment of the school.
        
        Returns:
            shaped_enroll(float): expected feeder enrollment from one school.
    
    """
    min_walk = 3 # minimum km walking distance
    max_walk = 5 # maximum km walking distance
    shaped_enroll = np.where(distance < min_walk, enrollment,
             np.where(distance>max_walk, 0, enrollment*(1-(distance-min_walk)/(max_walk-min_walk))))
    return shaped_enroll

def generate_random_sp(bounds):
    """  Function to generate a vector of 40000 random (lat, lon) starting points for each 
         proposed school within box boundary.
    """
    lat = np.random.uniform(low=bounds[0][0], high=bounds[1][0], size=40000)
    lon = np.random.uniform(low=bounds[0][1], high=bounds[1][1], size=40000)
    sp = np.vstack((lat, lon)).T
    return sp

def check_region(vec, region_shp):
    """ Function to check if initial vector of starting points are located within the regional boundaries/polygon. 
        Returns boolean vector
    """
    vec = gpd.points_from_xy(vec[:, 1], vec[:, 0]) # lat = y, x=lon
    return vec.within(region_shp[0])

def generate_sp_proposed(sp_list, proposed_schools):
    """ Returns a vector of random starting points of size (proposed_schools)
    """
    return sp_list[np.random.choice(sp_list.shape[0], proposed_schools,replace=False)]

def get_bounds(df_):
    """ Returns the minimum and maximum box boundary for school construction.
    """
    bounds = np.array([[np.min(df_[:,0]), np.min(df_[:,1])], [np.max(df_[:,0]), np.max(df_[:,1])]])
    return bounds

def get_cma_bounds(bounds, n):
    """ CMA-ES expects a list of size 2 for bounds. This function returns the bounds reshaped for CMA-ES
    """
    x1y1 = np.repeat([bounds[0,:]],n, axis=0).flatten()
    x2y2 = np.repeat([bounds[1,:]],n, axis=0).flatten()
    boundsxy = [x1y1,x2y2]
    return boundsxy

def create_results_table():
    """ Function that checks to see if a results csv exists, it creates the csv.
    """
    if os.path.isfile('./results.csv') == False:
        results = pd.DataFrame(columns=['region','maxits', 'proposed_schools_n','starting_point', 'algorithm','std', 'ee',
                                        'eei', 'proposed_locations', 'time', 'sigma'])
        results.to_csv('results.csv', index=False)

# Data Preparation

To maximise efficiency, data is further prepared. Most experiments use numpy arrays which are faster and use less memory.

In [384]:
def prepare_datasets(df_, region):
    """ A function which prepares MS and SS datasets for the experiments.
    
        Parameters: 
            df_ (pd Dataset): a pandas dataframe
            region (str): specified region to prepare.
            
        Returns array of prepared datasets: 
            [0] (pd Dataset): df_middle: Dataset filtered to only middle schools. 
            [1] (np float64) df_middle_enroll: Vector of MS enrollment data. Only the last two grades.
            [2]: (np float64) df_middle_loc: 2d vector of MS location data- lat lon point data. 
            [3]: (np float64) df_sec_enroll: Vector of SS enrollment data. Only grades 9 and 10 enrollment.
            [4]: (np float64) df_sec_loc: 2d vector of SS location data- lat lon point data. 
            [5]: (np float64) current_ms_distance: Vector of existing distances from MS to SS
    """
    
    df_ = df_.loc[df_['ADM1_EN'] == region] # filter dataset by region
    df_middle = df_.loc[df_['grade_7_8'] > 0] # 1
    df_middle_enroll = df_middle['grade_7_8'].reset_index(drop=True).to_numpy(dtype=float) # 2
    df_middle_loc = df_middle['point'].reset_index(drop=True).to_numpy()
    df_middle_loc = np.array([np.array(i) for i in df_middle_loc], dtype=float) # 3
    df_sec = df_.loc[ (df_['gr_offer'] == 'G. 9-10') | (df_['gr_offer'] == 'G. 9-12')]
    df_sec_enroll = df_sec['grade9_10'].reset_index(drop=True).to_numpy(dtype=float) # 4
    df_sec_loc = df_sec['point'].reset_index(drop=True).to_numpy() 
    df_sec_loc = np.array([np.array(i) for i in df_sec_loc], dtype=float) # 5
    current_ms_distance = df_middle['nearest_lwr_sec'].to_numpy() # 6
    return [df_middle, df_middle_enroll, df_middle_loc, df_sec_enroll, df_sec_loc, current_ms_distance]

In [393]:
# Use this for CMA. 
def find_eligible_MS(data, n, min_dict):
    
    """ A function that finds n clusters of schools with enrolment > minimum capacity: 200.
        If n clusters are not found, it returns the max possible that meet the minimum capacity requirements. 
    
        Parameters:
            data: (PD dataset): initial school dataset.
            n (int): number of new SS to construct
            min_dict (dict): Dictionary of regions (str) as keys and values (ints) are equal to minimum distance expected.
            
        Returns array of prepared datasets: 
            [0] (PD dataset): cluster_schools: Dataset of groupings of schools. 
            [1] (str): region: name of region
            [2]: (int): max_possible: total number of SS that can be constructed, always <=n

    """
    
    # Step 1. Check is distance is > variable minimum distance by region outlined above.
    data = data[data['nearest_lwr_sec'] > data['ADM1_EN'].map(min_dict)]
    region = data['ADM1_EN'].unique()[0]

    # Step 2. Prepare key MS datasets. 
    df_middle = data.loc[data['grade_7_8'] > 0].reset_index(drop=True) # Filter to MS only
    df_middle_enroll = df_middle['grade_7_8'].reset_index(drop=True).to_numpy(dtype=float) # Convert MS Enrolment to np
    df_middle_loc = df_middle['point'].reset_index(drop=True).to_numpy()
    df_middle_loc = np.array([np.array(i) for i in df_middle_loc], dtype=float) # Convert MS locational data to np.
    
    # Step 3. Calculate distance from MS to all other MS and check if < minimum distance per region
    dist = haversine_vector(df_middle_loc,df_middle_loc, Unit.KILOMETERS, comb=True) < min_dict[region]
    # Step 4. Find MS enrolment if within 5km
    find_enrolment = np.where(dist, df_middle_enroll,0) 
    new_enroll_ss = find_enrolment # create a copy for manipulation
    
    # Step 5. Expected new SS must be < length of dist. e.g. If length_dist = 15, new SS must be less than 15.
    length_dist = len(np.sum(new_enroll_ss, axis=1)) 
    
    # Step 6. Iteratively find top n enrolment of schools within 5km of each other. n=expected_schools or length_dist
    # To ensure there is no double counting, find  first max, then subtract from array of new_enroll_ss.
    # Repeat until expected enrolment found for top n schools. 

    for i in range(min(n, length_dist)):
        expected_enrol = np.sum(new_enroll_ss, axis=1) 
        max_ee_index = np.argpartition(expected_enrol, -(i+1))[-(i+1):][-(i+1)]
        max_array = new_enroll_ss[max_ee_index]
        new_enroll_ss = (new_enroll_ss - new_enroll_ss[max_ee_index])
        new_enroll_ss = new_enroll_ss.clip(min=0)
        new_enroll_ss[max_ee_index] = max_array
        
    # Step 7. Find new schools above minimum capacity of 200.
    new_enroll_ss = new_enroll_ss[np.sum(new_enroll_ss, axis=1) > 200] # minimum capacity of 200
    max_possible = min(len(new_enroll_ss), n) # total new SS possible
    new_schools_ee = np.sum(new_enroll_ss, axis=1) #  total ee possible from new SS.
    new_schools_index = np.argpartition(new_schools_ee, -(max_possible))[-(max_possible):]

    # Step 8. Find index of all new schools included in the final sample & return MS dataframe with only these MS.
    cluster_schools = np.where(new_enroll_ss[new_schools_index])[1] 
    cluster_schools = df_middle.loc[cluster_schools]
    region = list(pd.unique(data['ADM1_EN']))[0]
    return cluster_schools, region, max_possible

In [386]:
def prepare_all_datasets(df_):
    """Function that takes in pd dataset, prepares regional datasets for experiments and returns dictionary 
       key=region, values=output of prepare_datasets()"""
    regions = set(df_['ADM1_EN'])
    datasets = {}
    for i in regions:
        datasets[i] = prepare_datasets(df_, i)
    return datasets

In [387]:
def prepare_all_eligible_n(df_, n, min_dict):
    """Function that takes in pd datasets, finds all eligible MS, prepares them for experiments 
       and returns a dictionary  dictionary.
       
    Parameters:
        df_: (pd dataframe): initial school dataset.
        n: (int) total number of SS to construct
        min_dict: (dict) recommended distance per region
    
    Returns
        datasets2: (dict): key= region, values=output of prepare_datasets()
        regions_mx_possible: (dict): region and max number of new SS possible, always <=n
    """
    regions = set(df_['ADM1_EN'])
    dataframes = [find_eligible_MS(df_[df_['ADM1_EN']==i], n, min_dict) for i in regions]
    regions_mx_possible = dict([ [i[1], i[2]] for i in dataframes]) # region & total new SS possible.
    revised_df = pd.concat([df_[0] for df_ in dataframes], ignore_index=False)
    datasets2 = {}
    for i in regions:
        datasets2[i] = prepare_datasets(revised_df, i)
    return datasets2, regions_mx_possible

In [388]:
def create_random_sps(region, data_bounds, gdf_region):
    gdf_region_shp = gdf_region.loc[gdf_region['ADM1_EN']==region]['geometry'].reset_index(drop=True)
    bounds = get_bounds(data_bounds) # bounds from MS locations
    sp = generate_random_sp(bounds)# Create a large sample of starting points
    sp = sp[check_region(sp, gdf_region_shp)] # only include points that are within regional boundaries.
    return sp

In [389]:
def update_dataset_random(df_, gdf_r):
    for i in df_:
        df_[i].append(create_random_sps(i, df_[i][2], gdf_r))

In [390]:
# Variable filtering of dataset based on max distance for inclusion. 
# This is a decision variable and should be set by the Government. If >5km, only MS 5km from SS are considered. 
# If below 2, distance is not such an issue for maximising enrolment. Other appraoches should be sought.

### TO DO: CREATE A FUNCTION THAT DOES THIS. 

min_dist_dict = { 
'Addis Ababa': 2, # Insufficient only 3 >5km: X, 9 MS > 3km: X, 21 > 2km: ✓
 'Amhara': 5, # 3376 MS > 5km: ✓
 'Benishangul Gumz': 5, # 204 MS > 5km: ✓
 'Dire Dawa': 5, # 29 MS > 5km: ✓
 'Harari': 3, # Insufficient only 10 MS >5km: X 21 > 3km: ✓
 'Oromia': 5, # 5037 MS > 5km: ✓
 'SNNP': 5, # 675 MS > 5km: ✓
 'Somali': 5, # 272 MS > 5km: ✓
 'Tigray': 5 # 686 MS > 5km: ✓
}

In [394]:
%%time 
# Create datasets for experiments. n=5, and n=10.
df_all = prepare_all_datasets(df)
df_5, new_ss_regions_5 = prepare_all_eligible_n(df, 5, min_dist_dict)
df_10, new_ss_regions_10 = prepare_all_eligible_n(df, 10, min_dist_dict)

Wall time: 5.71 s


In [395]:
new_ss_regions_5

{'Tigray': 5,
 'Dire Dawa': 4,
 'Amhara': 5,
 'Oromia': 5,
 'Somali': 5,
 'Addis Ababa': 5,
 'Harari': 2,
 'SNNP': 5,
 'Benishangul Gumz': 5}

In [396]:
new_ss_regions_10

{'Tigray': 10,
 'Dire Dawa': 4,
 'Amhara': 10,
 'Oromia': 10,
 'Somali': 10,
 'Addis Ababa': 10,
 'Harari': 2,
 'SNNP': 10,
 'Benishangul Gumz': 10}

In [397]:
# For two regions, they are not required for n=10, as there are not sufficient schools to meet minimum capacity.
del new_ss_regions_10['Dire Dawa']
del new_ss_regions_10['Harari']

In [398]:
%%time
update_dataset_random(df_all, gdf_region)

Wall time: 1min 46s


In [399]:
%%time
update_dataset_random(df_5, gdf_region)

Wall time: 1min 56s


In [400]:
%%time
update_dataset_random(df_10, gdf_region)

Wall time: 2min 3s


## Experiments

In [401]:
### FITNESS FUNCTION

def expected_enrollment(middle_loc, x, middle_enroll, current_dist, ee_old_constant, min_dict):

    """ The Fitness Function returns the overall total expected enrollment increase given the new school locations x. 

        The function calculates the distance between each new school in x and the current middle schools to determine 
        the expected enrollment per new school. If a new school proposed is closer than the current secondary school, 
        enrollment from that subtracted from the overall enrollment gains. 

        Parameters:
            middle_loc: a vector of longitude and latitude coordinates of all middle schools
            x: a vector longitude and latitude coordinates for the newly proposed secondary schools
            middle_enroll: a vector of current enrollment at each middle school
            current_distnce: a vector of the current distances between each middle school and secondary school.

        Returns:
            eei + ee_old (float): the expected enrollment increase from each school in x  (eei)
                                  + the current secondary enrollment (ee_old)

    """

    ee_old = ee_old_constant.copy() # Overall SS Enrollment
    d = haversine_vector(middle_loc,x, Unit.KILOMETERS, comb=True) # distance of MS to x.
    d_min = np.min(d, axis=0) # select only closest schools to avoid duplication. 

    d_index = np.argmin(d, axis=0) # index of min distance of MS to x
    d2 = np.where((d_min < min_dict) & (d_min < current_dist)) # limit to only schools < 5km and schools < current distance
    # Put into dataframe for manipulation in pandas
    # index 0 = x, index 1 = MS, index 2 = distance, index 3 = shaped enrollment 
    d3 = pd.DataFrame(np.vstack((d_index[d2], d2[0], d_min[d2], shape(d_min[d2], middle_enroll[d2[0]]))).T) 
    d32 = d3.loc[d3.groupby([1])[2].idxmin()] # find only nearby SS if MS is close to more than 1 SS.
    d32 = d32.groupby(0)[3].sum() # The overall shaped enrollment for each school in x.
    eei = np.sum(d32) # overall expected enrollment increase for each school in x.
    # Find MS enrollment of MS if close to old SS. 
    distance_current = np.sum(shape(current_dist[d2], middle_enroll[d2]))
    ee_old -= distance_current # subtract shaped enrollment from overall SS enrollment as new school is close
    return eei + ee_old # return overall expected enrollment + current secondary enrollment

In [402]:
def f2(x, n, ms_loc, ms_enrol, ms_dist, ee_old, min_dict): # modified fitness function for advanced benchmark. 
    """ The Objective Function returns the maximum expected enrollment.
    """
    x = x.reshape(n,2) # reshape for input into expected enrolment.
    t_case = expected_enrollment(ms_loc, x, ms_enrol, ms_dist, ee_old, min_dict) # run fitness function.
    return t_case*-1 # Multiply by -1 for maximising.

In [408]:
def establish_benchmarks(data, n, min_dict):
    # *** ESTABLISH BENCHMARK ****
    
    # Basic Benchmark
    # Step 1. Find all MS with distance > 5km. 
    # Step 2. Sort by enrollment and select n schools with the highest enrollment
    # Step 3. Sum the enrollment to assume that the new schools will only serve n schoools.
    # Step 4. Save the results to results.csv
    start_time_basic = time.time()
    df_ms, df_ms_enrol, df_ms_loc, current_ms_dist = data[0], data[1], data[2], data[5]
    region = df_ms['ADM1_EN'].unique()[0]
    ee_old_constant = np.sum(data[3])
    df_ms = df_ms[df_ms['nearest_lwr_sec'] > df_ms['ADM1_EN'].map(min_dict)]
    min_dict_dist = min_dict[region]
    dd = df_ms.sort_values(['grade_7_8'], ascending=False).head(n) # filter by number of SS to construct
    n = len(dd)
    benchmark = sum(dd['grade_7_8']) # sum the enrollment for a basic indication of expected enrollment
    basic_time = time.time() - start_time_basic

    # Advanced Benchmark
    # Since MS with the highest enrollment are likely to be close to other MS, apply the fitness function to these schools
    # Step 1. Assume x is the locations of the top n schools
    # Step 2. Run fitness Function on these schools. 
    start_time_adv = time.time()
    benchmark_loc = dd['point'].reset_index(drop=True).to_numpy()
    benchmark_loc = np.array([np.array(i) for i in benchmark_loc], dtype=float)
    
    # Run Benchmark on objecitve function. 
    benchmark_f = f2(benchmark_loc, n, df_ms_loc, df_ms_enrol, current_ms_dist, ee_old_constant, min_dict_dist) # Run Fitness Function
    # Store results
    create_results_table() # create table for results, if it doesn't already exist.
    results = pd.read_csv('results.csv')

    row1 = (pd.Series({'region': region, 'maxits':np.nan, 'proposed_schools_n': n,
                       'starting_point':np.nan, 'algorithm':'Basic Benchmark', 'std':np.nan,
                       'ee': benchmark + ee_old_constant, 'eei': benchmark,
                       'proposed_locations': benchmark_loc, 'time':np.nan, 'sigma':np.nan}))

    row2 = (pd.Series({'region': region, 'maxits':np.nan, 'proposed_schools_n': n,'starting_point': np.nan, 'std':np.nan,
                       'algorithm':'Advanced Benchmark', 'ee': abs(round(benchmark_f,0)),
                       'eei': round(abs(benchmark_f)-ee_old_constant,0), 
                        'proposed_locations': benchmark_loc, 'time':np.nan, 'sigma':np.nan}))

    results = results.append([row1,row2], ignore_index=True)
    results.to_csv('results.csv', index=False)

    print(region, 'n=', n, ', Basic Algorithm Time:', basic_time)
    print(region, 'n=', n, ', ', 'Advanced Algorithm Time:', time.time() - start_time_adv)

In [404]:
def random_search(f, max_iterations, sp, n):
    """ Random Search Algorithm. Generates n random samples for max_iterations and applies fitness function and finds
        maximum expected enrolment.
        
        Parameters:
            f(): fitness function
            max_iterations (int): Maximum iterations
            sp(2d numpy array): starting point solution
            n(int): number of new schools to construct.
        
        Returns:
            best_f() (float): maximum expected enrolment
            best_solution: the longitude and latitude coordinates associated with the best f() value found.
    """
    x = [generate_sp_proposed(sp, n) for _ in range(max_iterations)]
    fx = [[f(xi), xi] for xi in x]
    best_f, best_solution = min(fx, key=lambda x:x[0])
    return best_f, best_solution

In [411]:
def run_random_search(n, n_sp, max_iterations, data, min_dict):
    """ Function which runs the random search algorithm.
    
        Parameters:
            max_iterations (int): Maximum iterations
            n_sp(2d numpy array): total number of starting points e.g. 30
            n(int): number of new schools to construct.
            data(dict): dictionary of prepared datasets.
        Returns: None
            New csv created with results stored: results.csv 
    """
    start_time_rs = time.time()
    df_ms, df_ms_enrol, df_ms_loc, current_ms_dist, sp = data[0], data[1], data[2], data[5], data[6]
    ee_old_constant = np.sum(data[3])
    region = df_ms['ADM1_EN'].unique()[0]
    min_dict_dist = min_dict[region]
    
    def f(x):
        """ The Objective Function for Random Search.
        """
        x = x.reshape(n,2) # reshape for input into expected enrolment.
        test_case = expected_enrollment(df_ms_loc, x, df_ms_enrol, current_ms_dist, ee_old_constant, min_dict_dist) # run fitness function.
        return test_case*-1 # Multiply by -1 for maximising.
    
    fx = []
    for _ in range(n_sp):
        start_time = time.time()
        fx.append([random_search(f, max_iterations, sp, n), time.time() - start_time])
    
    create_results_table() # create table for results, if it doesn't already exist.
    results = pd.read_csv('results.csv')

    for i in range(0, len(fx)):
        row = (pd.Series({'region': region, 'maxits':max_iterations, 'proposed_schools_n': n, 'std':np.nan,
                        'starting_point':i, 'algorithm':'Random Search', 'ee':round(abs(fx[i][0][0]),0),
                                    'eei':round(abs(ee_old_constant - abs(fx[i][0][0])),0), 
                                    'proposed_locations': fx[i][0][1], 'time':fx[i][1], 'sigma':'NA'}))
        results = results.append(row, ignore_index=True)
    results.to_csv('results.csv', index=False)

    print(region, 'n=', n, ', RS complete. Total time:', round(time.time() - start_time_rs,2))

In [464]:
def cma_region(n, n_starting_points, max_iterations, data, data_sub, min_dict_dist, set_sigma=None):
    
    """ Covariance Matrix Adaptation Evolution Strategy Algorithm (CMA-ES)
    
        Parameters:
            n: Number of SS to construct
            n_starting_points: Number of different starting points
            max_iterations: maximum iterations
            data: Datasets in the format of the output of the prepare_all_datasets() function.
            data_sub: Filtered Dataset in the format of the output of prepare_all_eligible_n() function
            set_sigma: Set sigma value or std for experiment. Default is None. If none, it will try 10 values from std.
        
        Returns: None
            New csv created with results stored: results.csv 
    
    """
    if len(data) == len(data_sub):
        algorithm = 'CMA-ES - 1'
    else:
        algorithm = 'CMA-ES - 2'
    start_time_cma = time.time()
    df_ms_sub, df_ms_enrol_sub, df_ms_loc_sub, current_ms_dist_sub = data_sub[0], data_sub[1], data_sub[2], data_sub[5]
    sp = data_sub[6] # random starting points
    ee_old_constant = np.sum(data[3]) # secondary school total
    region = df_ms_sub['ADM1_EN'].unique()[0]
    bounds = get_bounds(sp)
    cma_bounds = get_cma_bounds(bounds, n)

    def f(x):
        """ The Objective Function returns the maximum expected enrollment.
        """
        x = x.reshape(n,2) # reshape for input into expected enrolment.
        ee = expected_enrollment(df_ms_loc_sub, x, df_ms_enrol_sub, current_ms_dist_sub, ee_old_constant, min_dict_dist) # fitness function.
        return ee*-1 # Multiply by -1 for maximising.
       
    # Sigma is the initial standard deviation. Problem variables need to be scaled per region.
    # Square root of variance is std.
    sigma0 = np.sqrt(np.std(df_ms_loc_sub[:,0])**2 + np.std(df_ms_loc_sub[:,1])**2)
    
    if set_sigma:
        sigmas = [set_sigma]
    else:
        sigmas = np.linspace(0, sigma0, 11)[1:] # Take 10 values evenly spaced out between 0 to sigma0
    
    start_time_cma = time.time()
    fcma = []
    for i in range(n_starting_points):
        start_time = time.time()
        for j in sigmas:
            es = cma.CMAEvolutionStrategy(generate_sp_proposed(sp, n).flatten(), sigma0=j,
                                      inopts={'bounds': cma_bounds,'seed':1234})
            es.optimize(f, iterations=(max_iterations/ es.popsize))
            fcma.append((es.result[1], es.result[0].reshape(n, 2), (time.time() - start_time), j, i))
    
    create_results_table() # create table for results, if it doesn't already exist.
    results = pd.read_csv('results.csv')
    
    for i in range(0, len(fcma)):
        row = (pd.Series({'region': region, 'maxits':max_iterations,'proposed_schools_n': n,'std':sigma0,
                    'starting_point':fcma[i][4], 'algorithm':algorithm, 'ee':round(abs(fcma[i][0]),0),
                                'eei':round(abs(ee_old_constant - abs(fcma[i][0])),0), 
                                'proposed_locations': fcma[i][1], 'time':fcma[i][2], 'sigma':fcma[i][3]}))
        results = results.append(row, ignore_index=True)
    
    results.to_csv('results.csv', index=False)
    print(region, 'CMA complete. Total time:', round(time.time() - start_time_cma,2))

# Experiments

## How to:

1. Indicte the number of random starting points. 
2. Set the maximum iterations 
3. Loop through the keys of df_all- which is a dictionary of the prepared datasets. 
4. run establish_benchmark(). 
5. run_random_search()

- df_all = dictionary of prepared datasets
- new_ss_regions_5 = total number of SS that can be constructed. Max=5
- min_dist_dict = the dictionary of recommended minimum distance per region

Wall time refers to the overall time it has taken to execute the entire cell.

#### 1. Establish Basic and Advanced Benchmarks

In [423]:
%%time
# Benchmarks for n<=5
for i in new_ss_regions_5.keys():
    establish_benchmarks(df_all[i], new_ss_regions_5[i], min_dist_dict)

Tigray n= 5 , Basic Algorithm Time: 0.002973794937133789
Tigray n= 5 ,  Advanced Algorithm Time: 0.019999265670776367
Dire Dawa n= 4 , Basic Algorithm Time: 0.001997232437133789
Dire Dawa n= 4 ,  Advanced Algorithm Time: 0.023999929428100586
Amhara n= 5 , Basic Algorithm Time: 0.003999948501586914
Amhara n= 5 ,  Advanced Algorithm Time: 0.027001619338989258
Oromia n= 5 , Basic Algorithm Time: 0.005997657775878906
Oromia n= 5 ,  Advanced Algorithm Time: 0.028999805450439453
Somali n= 5 , Basic Algorithm Time: 0.0009996891021728516
Somali n= 5 ,  Advanced Algorithm Time: 0.02300119400024414
Addis Ababa n= 5 , Basic Algorithm Time: 0.0019989013671875
Addis Ababa n= 5 ,  Advanced Algorithm Time: 0.022039413452148438
Harari n= 2 , Basic Algorithm Time: 0.0009620189666748047
Harari n= 2 ,  Advanced Algorithm Time: 0.022999048233032227
SNNP n= 5 , Basic Algorithm Time: 0.002001047134399414
SNNP n= 5 ,  Advanced Algorithm Time: 0.0230405330657959
Benishangul Gumz n= 5 , Basic Algorithm Time: 0

In [424]:
%%time
# Benchmarks for n<=10 
for i in new_ss_regions_10.keys():
    establish_benchmarks(df_all[i], new_ss_regions_10[i], min_dist_dict)

Tigray n= 10 , Basic Algorithm Time: 0.0020012855529785156
Tigray n= 10 ,  Advanced Algorithm Time: 0.028045654296875
Amhara n= 10 , Basic Algorithm Time: 0.003979921340942383
Amhara n= 10 ,  Advanced Algorithm Time: 0.0329742431640625
Oromia n= 10 , Basic Algorithm Time: 0.006998538970947266
Oromia n= 10 ,  Advanced Algorithm Time: 0.031000852584838867
Somali n= 10 , Basic Algorithm Time: 0.003000497817993164
Somali n= 10 ,  Advanced Algorithm Time: 0.02705097198486328
Addis Ababa n= 10 , Basic Algorithm Time: 0.001947164535522461
Addis Ababa n= 10 ,  Advanced Algorithm Time: 0.025037050247192383
SNNP n= 10 , Basic Algorithm Time: 0.002966642379760742
SNNP n= 10 ,  Advanced Algorithm Time: 0.026035070419311523
Benishangul Gumz n= 10 , Basic Algorithm Time: 0.0010008811950683594
Benishangul Gumz n= 10 ,  Advanced Algorithm Time: 0.02399897575378418
Wall time: 216 ms


#### 2. Run random search (maxits=20,000, n=5, n=10)

In [430]:
%%time 
n_starting_points = 30
maxits= 20000
# for maximum of 10. 
for i in new_ss_regions_5.keys():
    run_random_search(new_ss_regions_5[i], n_starting_points, maxits, df_all[i], min_dist_dict)

Tigray n= 5 , RS complete. Total time: 1765.8
Dire Dawa n= 4 , RS complete. Total time: 1728.43
Amhara n= 5 , RS complete. Total time: 2156.19
Oromia n= 5 , RS complete. Total time: 3088.16
Somali n= 5 , RS complete. Total time: 1484.19
Addis Ababa n= 5 , RS complete. Total time: 1488.53
Harari n= 2 , RS complete. Total time: 1473.96
SNNP n= 5 , RS complete. Total time: 1747.86
Benishangul Gumz n= 5 , RS complete. Total time: 1462.5
Wall time: 4h 33min 15s


In [431]:
%%time 
n_starting_points = 3043
maxits= 20000
# for maximum of 10. 
for i in new_ss_regions_10.keys():
    run_random_search(new_ss_regions_10[i], n_starting_points, maxits, df_all[i], min_dist_dict)  

Tigray n= 10 , RS complete. Total time: 2335.63
Amhara n= 10 , RS complete. Total time: 3077.75
Oromia n= 10 , RS complete. Total time: 4007.96
Somali n= 10 , RS complete. Total time: 1338.55
Addis Ababa n= 10 , RS complete. Total time: 2255.85
SNNP n= 10 , RS complete. Total time: 2232.66
Benishangul Gumz n= 10 , RS complete. Total time: 1644.77
Wall time: 4h 41min 33s


#### 3. Run CMA-ES-2 (maxits=20,000, n=5)

## How to:

CMA-ES-1 can be run by not using the filtered dataset e.g. See the format below:

* cma_region(new_ss_regions_5[i], n_starting_points, maxits, <b> df_all[i], df_all[i] <b>)

CMA-ES-2 can be run by using the filtered dataset e.g:  see the format below:

* cma_region(new_ss_regions_5[i], n_starting_points, maxits, <b> df_all[i], df_5[i] <b>)

In [445]:
%%time
n_starting_points = 30
maxits= 20000
# for maximum of 5. 
for i in df_all.keys():
    cma_region(new_ss_regions_5[i], n_starting_points, maxits, df_all[i], df_5[i], min_dist_dict[i])

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 11:05:00 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.214210000000000e+05 1.0e+00 5.52e-02  5e-02  6e-02 0:00.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 11:05:00 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.214210000000000e+05 1.0e+00 1.10e-01  1e-01  1e-01 0:00.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 11:05:00 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.214210000000000e+05 1.0e+00 1.66e-01  2e-01  2e-01 0:00.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 11:05:00 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.222650000000000e+05 1.0e+00 2.23e-01  2e-01  2e-01 0:00.0
    2     20 -1.222650000000000e+05 1.2e+00 2.21e-01  2e

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 11:07:17 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.234725659643909e+05 1.0e+00 3.16e-01  3e-01  3e-01 0:00.0
    2     20 -1.232124609635058e+05 1.2e+00 3.20e-01  3e-01  3e-01 0:00.0
    3     30 -1.238388618282351e+05 1.2e+00 3.22e-01  3e-01  3e-01 0:00.0
  100   1000 -1.250537317393730e+05 1.5e+01 4.66e-02  6e-03  5e-02 0:02.3
  132   1320 -1.251570000000000e+05 3.1e+01 6.95e-02  5e-03  9e-02 0:03.2
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 11:07:21 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.214210000000000e+05 1.0e+00 3.86e-01  4e-01  4e-01 0:00.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 11:07:21 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.227079776421747e+05 1.0e+00 4.47e-01  4e-01  5e-01 0:0

KeyboardInterrupt: 

#### 5. Run CMA-ES-1 (maxits=20,000, n=5)

In [443]:
%%time
n_starting_points = 30
maxits= 20000
# for maximum of 5. 
for i in df_all.keys():
    cma_region(new_ss_regions_5[i], n_starting_points, maxits, df_all[i], df_all[i], min_dist_dict[i])

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Tue Sep  6 10:38:28 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.219349788879924e+05 1.0e+00 9.53e-02  9e-02  1e-01 0:00.0
    2     20 -1.222971899569808e+05 1.3e+00 9.14e-02  9e-02  1e-01 0:00.0
    3     30 -1.223847741514681e+05 1.3e+00 8.44e-02  8e-02  9e-02 0:00.1
  100   1000 -1.237016329310183e+05 6.4e+00 2.16e-03  5e-04  3e-03 0:02.5
  200   2000 -1.237027087360525e+05 1.4e+02 1.07e-04  1e-05  2e-04 0:05.5
  300   3000 -1.237027097185529e+05 4.2e+03 8.59e-06  6e-07  1e-05 0:08.3
  400   4000 -1.237027097210406e+05 1.4e+05 1.18e-06  3e-08  1e-06 0:11.0


c:\users\darra\appdata\local\programs\python\python38\lib\site-packages\cma\utilities\utils.py:343: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.1e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (class=CMAEvolutionStrategy method=alleviate_conditioning iteration=456)
  warnings.warn(msg + ' (' +


  500   5000 -1.237027097210194e+05 3.0e+00 1.98e-07  1e-07  2e-07 0:13.7
  600   6000 -1.237027097210731e+05 8.8e+00 8.00e-07  5e-07  1e-06 0:16.5
  700   7000 -1.237027097210744e+05 1.2e+01 2.47e-07  1e-07  3e-07 0:19.4
  800   8000 -1.237027097210728e+05 1.9e+01 9.83e-07  4e-07  1e-06 0:22.2
  900   9000 -1.237027097210776e+05 3.4e+01 1.92e-06  5e-07  2e-06 0:25.1
  990   9900 -1.237027097210679e+05 7.2e+01 9.03e-07  2e-07  1e-06 0:28.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Tue Sep  6 10:38:58 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.217804548301928e+05 1.0e+00 1.69e-01  2e-01  2e-01 0:00.0
    2     20 -1.219600000000000e+05 1.2e+00 1.57e-01  1e-01  2e-01 0:00.0
    3     30 -1.218735266679198e+05 1.2e+00 1.46e-01  1e-01  1e-01 0:00.0
  100   1000 -1.239673021838907e+05 6.4e+00 7.57e-03  2e-03  8e-03 0:02.6
  200   2000 -1.239883156827908e+05 4.9e+01 7.85e-05  4e-06  1e-04 0:05.7
  300   3000 -1.23988324

c:\users\darra\appdata\local\programs\python\python38\lib\site-packages\cma\utilities\utils.py:343: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.0e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (class=CMAEvolutionStrategy method=alleviate_conditioning iteration=975)
  warnings.warn(msg + ' (' +


  100   1000 -1.243077454645686e+05 4.0e+00 2.51e-01  2e-01  3e-01 0:02.4
  200   2000 -1.276656552853511e+05 6.9e+00 4.54e-02  7e-03  4e-02 0:05.4
  300   3000 -1.277398687002005e+05 9.2e+01 1.61e-03  4e-05  1e-03 0:09.5
  400   4000 -1.277399686683778e+05 8.6e+02 2.43e-04  1e-06  3e-04 0:13.5
  500   5000 -1.277399708856703e+05 1.1e+04 3.18e-05  2e-08  4e-05 0:17.6
NOTE (module=cma, iteration=556):  
condition in coordinate system exceeded 1.0e+08, rescaled to 1.0e+00, 
condition changed from 1.6e+09 to 3.4e+06
  600   6000 -1.277399709222093e+05 4.3e+03 1.51e-06  9e-11  2e-06 0:22.6
  700   7000 -1.277399709224267e+05 6.7e+04 1.42e-07  9e-13  3e-07 0:27.0
  780   7800 -1.277399709224281e+05 4.7e+05 1.87e-08  2e-14  5e-08 0:30.8
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Tue Sep  6 10:40:37 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.223917462030829e+05 1.0e+00 5.01e-01  5e-01  5e-01 0:00.0
    2     20 -1.2243338

...

    2     20 -2.427462892633471e+04 1.2e+00 9.94e-01  9e-01  1e+00 0:00.0
    3     30 -2.431141056164600e+04 1.3e+00 1.03e+00  1e+00  1e+00 0:00.1
  100   1000 -2.434627957333249e+04 6.2e+00 2.51e+00  2e+00  4e+00 0:01.7
  200   2000 -2.431373075488154e+04 1.8e+01 2.03e+00  1e+00  4e+00 0:03.4
  300   3000 -2.435844291628525e+04 4.3e+01 1.60e+00  5e-01  3e+00 0:05.1
  400   4000 -2.453901798199111e+04 6.3e+01 1.54e+00  6e-01  2e+00 0:06.8
  500   5000 -2.502725622481324e+04 6.7e+01 4.28e-01  8e-02  3e-01 0:08.5
  600   6000 -2.565281907089266e+04 3.4e+01 9.43e-03  5e-04  3e-03 0:10.8
  700   7000 -2.565498913658670e+04 9.6e+01 3.90e-04  3e-06  2e-04 0:13.3
  800   8000 -2.565499140307567e+04 1.3e+03 7.53e-05  4e-08  3e-05 0:15.8
NOTE (module=cma, iteration=872):  
condition in coordinate system exceeded 1.0e+08, rescaled to 1.0e+00, 
condition changed from 2.3e+08 to 1.4e+04
  900   9000 -2.565499145178797e+04 1.4e+02 8.07e-06  2e-10  6e-06 0:18.3
 1000  10000 -2.565499145203647e+04 1

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.446429240224063e+04 1.0e+00 7.59e-01  7e-01  8e-01 0:00.0
    2     20 -2.433142758575070e+04 1.2e+00 7.37e-01  7e-01  8e-01 0:00.0
    3     30 -2.450084253811635e+04 1.2e+00 6.89e-01  6e-01  7e-01 0:00.1
  100   1000 -2.521034183261788e+04 5.2e+00 7.71e-02  3e-02  8e-02 0:02.0
  200   2000 -2.565212367579744e+04 9.3e+00 2.70e-03  3e-04  2e-03 0:04.4
  300   3000 -2.565399059376071e+04 1.5e+02 4.52e-05  3e-07  4e-05 0:06.7
  400   4000 -2.565399145147728e+04 2.6e+03 1.80e-06  8e-10  1e-06 0:09.1
NOTE (module=cma, iteration=463):  
condition in coordinate system exceeded 1.1e+08, rescaled to 1.0e+00, 
condition changed from 2.0e+08 to 6.2e+01
  500   5000 -2.565399145202615e+04 7.9e+00 1.10e-07  3e-12  1e-07 0:11.5
  600   6000 -2.565399145203686e+04 1.2e+02 1.28e-08  2e-14  1e-08 0:13.8
  647   6470 -2.565399145203688e+04 5.6e+02 1.15e-08  6e-15  2e-08 0:15.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in 

  300   3000 -2.531179235525293e+04 1.7e+02 4.20e-05  3e-06  3e-05 0:06.6
  400   4000 -2.531179275935882e+04 2.8e+03 4.89e-06  2e-07  5e-06 0:09.0
  500   5000 -2.531179276070787e+04 5.3e+04 2.40e-07  8e-09  3e-07 0:11.5
  600   6000 -2.531179276072470e+04 6.1e+05 1.24e-08  4e-10  1e-08 0:13.9
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 02:33:41 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.445200000000000e+04 1.0e+00 7.88e-01  7e-01  9e-01 0:00.0
    2     20 -2.441196137094152e+04 1.3e+00 7.72e-01  7e-01  8e-01 0:00.0
    3     30 -2.456131366462585e+04 1.3e+00 7.10e-01  6e-01  8e-01 0:00.1
  100   1000 -2.506204399415607e+04 4.7e+00 1.08e-01  4e-02  1e-01 0:01.9
  200   2000 -2.613400000000000e+04 5.0e+00 6.44e-03  1e-03  6e-03 0:04.5
  235   2350 -2.613400000000000e+04 1.3e+01 8.85e-04  6e-05  7e-04 0:05.4
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 02:33:47 2022)
Iterat #

  800   8000 -2.576378935917407e+04 7.8e+01 3.20e-06  8e-07  7e-06 0:19.2
  900   9000 -2.576378935917377e+04 1.9e+02 6.29e-06  1e-06  2e-05 0:21.6
 1000  10000 -2.576378935917465e+04 2.6e+02 2.79e-06  5e-07  8e-06 0:24.1
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  11000 -2.576378935917365e+04 3.7e+02 1.35e-06  2e-07  4e-06 0:26.7
 1200  12000 -2.576378935917453e+04 4.3e+02 4.00e-06  8e-07  1e-05 0:29.4
 1210  12100 -2.576378935917259e+04 4.5e+02 2.88e-06  5e-07  8e-06 0:29.7
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 02:35:53 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.442960073057543e+04 1.0e+00 7.82e-01  8e-01  8e-01 0:00.0
    2     20 -2.449410479515583e+04 1.2e+00 7.27e-01  7e-01  8e-01 0:00.0
    3     30 -2.443600820224942e+04 1.3e+00 6.72e-01  6e-01  7e-01 0:00.1
  100   1000 -2.501860045064536e+04 4.0e+00 1.09e-01  6e-02  1e-01 0:01.9
  200   2000 -2.5252254098

  600   6000 -2.531672874121094e+04 1.0e+01 2.56e-06  1e-06  5e-06 0:14.8
  700   7000 -2.531672874121122e+04 1.7e+01 4.66e-06  2e-06  8e-06 0:17.3
  800   8000 -2.531672874121018e+04 3.0e+01 1.04e-05  4e-06  2e-05 0:20.0
  865   8650 -2.531672874121067e+04 4.1e+01 6.30e-06  2e-06  1e-05 0:21.8
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 02:37:49 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.458200000000000e+04 1.0e+00 7.17e-01  7e-01  7e-01 0:00.0
    2     20 -2.461400000000000e+04 1.1e+00 6.99e-01  7e-01  7e-01 0:00.0
    3     30 -2.441480350455661e+04 1.2e+00 7.27e-01  7e-01  8e-01 0:00.1
  100   1000 -2.528534636920522e+04 3.7e+00 1.06e-01  5e-02  1e-01 0:01.8
  200   2000 -2.574168964302271e+04 1.0e+01 1.06e-03  1e-04  1e-03 0:04.7
  300   3000 -2.574252133700805e+04 9.6e+01 3.08e-05  1e-06  4e-05 0:07.5
  400   4000 -2.574256056186665e+04 4.0e+02 1.20e-05  9e-08  2e-05 0:10.1
  500   5000 -2.57425611

#### 7. Find best sigma values, run CMA-ES- 2 with only the best sigma values (maxits=30,000, n=5)

In [450]:
# Open best sigma values and re-run experiment with maxits=40,000
b_sigma1 = pd.read_csv('best_sigma_cma1.csv') 
b_sigma2 = pd.read_csv('best_sigma_cma2.csv') 
best_sigma1 = dict(zip(b_sigma1['region'], b_sigma1['sigma'])) 
best_sigma2 = dict(zip(b_sigma2['region'], b_sigma2['sigma'])) 

In [461]:
%%time
n_starting_points = 30
maxits= 30000
# for maximum of 5. 
for i in df_all.keys():
    cma_region(new_ss_regions_5[i], n_starting_points, maxits, df_all[i], df_5[i], min_dist_dict[i],
               set_sigma=best_sigma2[i])

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 13:55:03 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.214210000000000e+05 1.0e+00 5.52e-01  5e-01  6e-01 0:00.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 13:55:03 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.227400000000000e+05 1.0e+00 5.62e-01  5e-01  6e-01 0:00.0
    2     20 -1.229489657053565e+05 1.2e+00 5.76e-01  5e-01  6e-01 0:00.0
    3     30 -1.225450258944336e+05 1.3e+00 6.00e-01  6e-01  6e-01 0:00.1
   85    850 -1.251570000000000e+05 1.1e+01 3.00e-02  5e-03  3e-02 0:02.1
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 13:55:05 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.221711585026925e+05 1.0e+00 5.69e-01  5e-01  6e-01 0:00.0
    2     20 -1.222356507055571e+05 1.2e+00 5.56e-01  5e-01  6e-01 0:0

c:\users\darra\appdata\local\programs\python\python38\lib\site-packages\cma\utilities\utils.py:343: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.1e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (class=CMAEvolutionStrategy method=alleviate_conditioning iteration=514)
  warnings.warn(msg + ' (' +


  600   6000 -1.260392977592555e+05 5.8e+00 3.12e-07  2e-07  4e-07 0:20.6
  700   7000 -1.260392977592693e+05 1.4e+01 2.51e-07  1e-07  4e-07 0:24.8
  750   7500 -1.260392977593099e+05 1.9e+01 5.58e-07  3e-07  8e-07 0:27.1
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 13:55:33 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.239742516453811e+05 1.0e+00 5.71e-01  5e-01  6e-01 0:00.0
    2     20 -1.243572604490135e+05 1.2e+00 5.66e-01  5e-01  6e-01 0:00.0
    3     30 -1.225922612844641e+05 1.3e+00 5.46e-01  5e-01  6e-01 0:00.1
  100   1000 -1.274262701985551e+05 7.8e+00 4.91e-02  1e-02  6e-02 0:02.6
  200   2000 -1.274955929265290e+05 4.6e+01 2.00e-03  1e-04  3e-03 0:06.2
  300   3000 -1.274956876432516e+05 9.5e+02 1.41e-04  1e-06  3e-04 0:09.8
  400   4000 -1.274956886720016e+05 1.4e+04 3.62e-05  2e-07  1e-04 0:13.6
  500   5000 -1.274956886877324e+05 2.0e+05 2.20e-06  1e-08  5e-06 0:17.6


c:\users\darra\appdata\local\programs\python\python38\lib\site-packages\cma\utilities\utils.py:343: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.2e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (class=CMAEvolutionStrategy method=alleviate_conditioning iteration=554)
  warnings.warn(msg + ' (' +


  600   6000 -1.274956886877621e+05 3.2e+00 2.19e-07  2e-07  3e-07 0:22.1
  700   7000 -1.274956886877654e+05 1.1e+01 1.21e-07  6e-08  2e-07 0:26.2
  800   8000 -1.274956886877619e+05 2.5e+01 6.55e-08  3e-08  1e-07 0:30.4
  900   9000 -1.274956886877684e+05 4.1e+01 9.87e-08  4e-08  1e-07 0:35.2
  980   9800 -1.274956886877644e+05 4.6e+01 6.29e-08  3e-08  7e-08 0:38.9
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 13:56:14 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.222092401497193e+05 1.0e+00 5.59e-01  5e-01  6e-01 0:00.0
    2     20 -1.225212779177305e+05 1.2e+00 5.23e-01  5e-01  5e-01 0:00.1
    3     30 -1.223883317098001e+05 1.3e+00 5.02e-01  5e-01  5e-01 0:00.1
  100   1000 -1.250224458521684e+05 5.3e+00 8.16e-02  2e-02  9e-02 0:03.0
  122   1220 -1.251570000000000e+05 9.0e+00 3.15e-02  5e-03  3e-02 0:04.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 13:56:18 2022)
Iterat #

...

Addis Ababa CMA complete. Total time: 5.68
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1234, Wed Sep  7 14:36:02 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -4.802591285305000e+03 1.0e+00 2.74e-02  3e-02  3e-02 0:00.0
    2     16 -4.802591285305000e+03 1.5e+00 2.63e-02  2e-02  3e-02 0:00.0
    3     24 -4.802591285305000e+03 1.5e+00 2.48e-02  2e-02  3e-02 0:00.1
    7     56 -4.802591285305000e+03 1.8e+00 1.50e-02  1e-02  1e-02 0:00.1
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1234, Wed Sep  7 14:36:02 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -4.580591285305000e+03 1.0e+00 2.61e-02  2e-02  3e-02 0:00.0
    2     16 -4.580591285305000e+03 1.3e+00 2.71e-02  2e-02  3e-02 0:00.0
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1234, Wed Sep  7 14:36:02 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -4.802591285305000e+

    1      8 -4.580591285305000e+03 1.0e+00 2.62e-02  2e-02  3e-02 0:00.0
    2     16 -4.580591285305000e+03 1.3e+00 2.73e-02  2e-02  3e-02 0:00.0
    3     24 -4.802591285305000e+03 1.8e+00 2.23e-02  2e-02  3e-02 0:00.0
   10     80 -4.580591285305000e+03 2.9e+00 1.29e-02  6e-03  1e-02 0:00.2
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1234, Wed Sep  7 14:36:07 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -4.580591285305000e+03 1.0e+00 2.88e-02  3e-02  3e-02 0:00.0
    2     16 -4.580591285305000e+03 1.4e+00 2.84e-02  3e-02  3e-02 0:00.0
    3     24 -4.580591285305000e+03 1.4e+00 2.76e-02  2e-02  3e-02 0:00.1
   19    152 -4.802591285305000e+03 2.8e+00 2.15e-02  1e-02  2e-02 0:00.4
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1234, Wed Sep  7 14:36:07 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -4.580591285305000e+03 1.0e+00 2.62e-02  2e-02  3e-02 0:00.0
    2     16 -

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.639370000000000e+05 1.0e+00 9.01e-02  9e-02  9e-02 0:00.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 14:36:11 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.639370000000000e+05 1.0e+00 9.01e-02  9e-02  9e-02 0:00.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 14:36:11 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.639370000000000e+05 1.0e+00 9.01e-02  9e-02  9e-02 0:00.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 14:36:11 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.639370000000000e+05 1.0e+00 9.01e-02  9e-02  9e-02 0:00.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 14:36:11 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&

    2     20 -2.465019785130183e+04 1.2e+00 2.85e-01  3e-01  3e-01 0:00.0
    3     30 -2.420000000000000e+04 1.2e+00 2.91e-01  3e-01  3e-01 0:00.0
    4     40 -2.420000000000000e+04 1.3e+00 2.89e-01  3e-01  3e-01 0:00.1
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 14:36:29 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.458200000000000e+04 1.0e+00 3.20e-01  3e-01  3e-01 0:00.0
    2     20 -2.461400000000000e+04 1.2e+00 3.03e-01  3e-01  3e-01 0:00.0
    3     30 -2.499600000000000e+04 1.2e+00 2.94e-01  3e-01  3e-01 0:00.1
  100   1000 -2.544900000000000e+04 4.5e+00 2.56e-02  8e-03  3e-02 0:02.7
  103   1030 -2.544900000000000e+04 4.7e+00 2.46e-02  7e-03  2e-02 0:02.8
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 14:36:32 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.453584891435233e+04 1.0e+00 3.27e-01  3e-01  3e-01 0:00.0
    2     

    2     20 -2.465019785130183e+04 1.2e+00 2.85e-01  3e-01  3e-01 0:00.0
    3     30 -2.420000000000000e+04 1.2e+00 2.91e-01  3e-01  3e-01 0:00.1
    4     40 -2.420000000000000e+04 1.3e+00 2.89e-01  3e-01  3e-01 0:00.1
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 14:37:07 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.458200000000000e+04 1.0e+00 3.20e-01  3e-01  3e-01 0:00.0
    2     20 -2.461400000000000e+04 1.2e+00 3.03e-01  3e-01  3e-01 0:00.0
    3     30 -2.499600000000000e+04 1.2e+00 2.94e-01  3e-01  3e-01 0:00.1
  100   1000 -2.544900000000000e+04 4.5e+00 2.56e-02  8e-03  3e-02 0:03.0
  103   1030 -2.544900000000000e+04 4.7e+00 2.46e-02  7e-03  2e-02 0:03.0
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 14:37:11 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.453584891435233e+04 1.0e+00 3.27e-01  3e-01  3e-01 0:00.0
    2     

#### 8. Find best sigma values, run CMA-ES- 1 with only the best sigma values (maxits=30,000, n=5)

In [462]:
%%time
n_starting_points = 30
maxits= 30000
# for maximum of 5. 
for i in df_all.keys():
    cma_region(new_ss_regions_5[i], n_starting_points, maxits, df_all[i], df_all[i], min_dist_dict[i],
               set_sigma=best_sigma1[i])

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 14:37:11 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.221496500003133e+05 1.0e+00 7.32e-01  7e-01  8e-01 0:00.0
    2     20 -1.224360831425171e+05 1.2e+00 6.83e-01  7e-01  7e-01 0:00.1
    3     30 -1.219721184826450e+05 1.2e+00 6.30e-01  6e-01  7e-01 0:00.1
  100   1000 -1.240205264086593e+05 4.5e+00 2.12e-01  8e-02  2e-01 0:02.7
  198   1980 -1.277886142009568e+05 2.0e+01 6.63e-03  5e-04  7e-03 0:06.7
  200   2000 -1.277899169902823e+05 2.1e+01 6.46e-03  5e-04  7e-03 0:06.8
  300   3000 -1.277918353261375e+05 7.2e+02 4.58e-04  3e-06  7e-04 0:12.1
  400   4000 -1.277918388747422e+05 1.8e+04 7.52e-05  3e-08  1e-04 0:17.5
NOTE (module=cma, iteration=432):  
condition in coordinate system exceeded 1.1e+08, rescaled to 1.0e+00, 
condition changed from 2.3e+09 to 1.2e+02
  500   5000 -1.277918389021849e+05 7.0e+01 3.79e-05  7e-10  9e-05 0:22.4
  600   6000 -1.2779183

c:\users\darra\appdata\local\programs\python\python38\lib\site-packages\cma\utilities\utils.py:343: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.1e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (class=CMAEvolutionStrategy method=alleviate_conditioning iteration=1293)
  warnings.warn(msg + ' (' +


 1300  13000 -1.240943586771214e+05 1.9e+00 8.48e-07  7e-07  1e-06 0:43.4
 1320  13200 -1.240943586771327e+05 3.1e+00 1.01e-06  7e-07  1e-06 0:44.1
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 14:40:53 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -1.219397862901723e+05 1.0e+00 7.19e-01  7e-01  8e-01 0:00.0
    2     20 -1.223270630250252e+05 1.2e+00 6.86e-01  7e-01  7e-01 0:00.0
    3     30 -1.220541406303449e+05 1.3e+00 6.63e-01  6e-01  7e-01 0:00.1
  100   1000 -1.242005121503023e+05 3.5e+00 1.40e-02  5e-03  1e-02 0:02.9
  200   2000 -1.243404726899025e+05 2.6e+01 3.46e-04  6e-05  3e-04 0:05.9
  300   3000 -1.243405746177473e+05 2.2e+02 2.20e-05  1e-06  2e-05 0:09.0
  400   4000 -1.243405774171813e+05 1.3e+03 1.58e-06  2e-08  2e-06 0:12.0
  500   5000 -1.243405775073479e+05 1.1e+04 8.70e-07  3e-09  1e-06 0:15.0
  600   6000 -1.243405775116572e+05 6.0e+04 3.52e-08  2e-11  7e-08 0:18.1
NOTE (module=cma, iterat

    2     20 -1.236240640940606e+05 1.2e+00 6.41e-01  6e-01  7e-01 0:00.1
    3     30 -1.235662820918943e+05 1.2e+00 6.07e-01  6e-01  6e-01 0:00.1
  100   1000 -1.242522979404025e+05 4.8e+00 8.18e-03  2e-03  9e-03 0:02.9
  200   2000 -1.242948725049676e+05 6.0e+01 7.85e-04  3e-05  1e-03 0:06.4
  300   3000 -1.242953070657748e+05 4.8e+02 2.28e-05  1e-07  3e-05 0:09.8
  400   4000 -1.242953086176951e+05 5.2e+03 4.69e-06  1e-08  9e-06 0:13.3
  500   5000 -1.242953086374806e+05 3.8e+04 3.43e-07  1e-10  6e-07 0:16.6
NOTE (module=cma, iteration=522):  
condition in coordinate system exceeded 1.0e+08, rescaled to 1.0e+00, 
condition changed from 4.7e+09 to 9.6e+04
  600   6000 -1.242953086376507e+05 2.1e+03 3.09e-08  1e-12  7e-08 0:19.6
  700   7000 -1.242953086376532e+05 1.6e+04 7.78e-09  5e-14  3e-08 0:22.7
  800   8000 -1.242953086376531e+05 1.8e+04 1.08e-08  6e-14  3e-08 0:26.1
  900   9000 -1.242953086376535e+05 4.3e+04 2.01e-08  4e-14  1e-07 0:29.4
 1000  10000 -1.242953086376536e+05 1

c:\users\darra\appdata\local\programs\python\python38\lib\site-packages\cma\utilities\utils.py:343: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.0e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (class=CMAEvolutionStrategy method=alleviate_conditioning iteration=1159)
  warnings.warn(msg + ' (' +


...

  600   6000 -2.529678307100906e+04 8.4e+00 2.42e-07  1e-07  3e-07 0:42.2
  700   7000 -2.529678307100973e+04 8.2e+01 8.80e-08  3e-08  2e-07 0:49.4
  800   8000 -2.529678307100977e+04 1.6e+02 6.09e-08  2e-08  9e-08 0:56.9
  900   9000 -2.529678307100977e+04 4.8e+02 2.88e-08  8e-09  7e-08 1:03.4
 1000  10000 -2.529678307100951e+04 4.6e+02 1.87e-07  5e-08  4e-07 1:09.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1075  10750 -2.529678307100972e+04 7.8e+02 1.42e-07  3e-08  3e-07 1:14.7
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 17:13:28 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.461751516167731e+04 1.0e+00 8.08e-01  8e-01  9e-01 0:00.1
    2     20 -2.453006162132842e+04 1.2e+00 7.82e-01  7e-01  9e-01 0:00.1
    3     30 -2.432000000000000e+04 1.3e+00 6.95e-01  6e-01  8e-01 0:00.1
   69    690 -2.452332439005902e+04 4.3e+00 6.20e-01  4e-01  7e-01 0:03.2
  100   1000 -2.4356698230

c:\users\darra\appdata\local\programs\python\python38\lib\site-packages\cma\utilities\utils.py:343: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.2e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (class=CMAEvolutionStrategy method=alleviate_conditioning iteration=539)
  warnings.warn(msg + ' (' +


  600   6000 -2.562566534698866e+04 1.3e+01 2.07e-05  4e-06  2e-05 0:41.8
  700   7000 -2.562566534698927e+04 2.2e+02 1.06e-05  2e-06  2e-05 0:49.9
  800   8000 -2.562566534698921e+04 7.4e+02 5.24e-06  8e-07  9e-06 0:57.6
  900   9000 -2.562566534698926e+04 1.5e+03 5.98e-06  6e-07  1e-05 1:06.5
 1000  10000 -2.562566534698927e+04 3.0e+03 1.50e-05  2e-06  3e-05 1:15.0
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1075  10750 -2.562566534698924e+04 3.1e+03 2.35e-05  3e-06  3e-05 1:20.8
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 17:18:41 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.461751516167731e+04 1.0e+00 8.08e-01  8e-01  9e-01 0:00.1
    2     20 -2.453006162132842e+04 1.2e+00 7.82e-01  7e-01  9e-01 0:00.1
    3     30 -2.432000000000000e+04 1.3e+00 6.95e-01  6e-01  8e-01 0:00.2
   62    620 -2.455251317192980e+04 3.7e+00 4.54e-01  3e-01  5e-01 0:03.2
  100   1000 -2.4356698230

  200   2000 -2.434200000000000e+04 9.6e+00 3.76e-01  1e-01  4e-01 0:12.1
  300   3000 -2.516402062402416e+04 1.6e+01 3.72e-02  6e-03  3e-02 0:18.2
  400   4000 -2.529407574952021e+04 2.4e+01 1.10e-03  4e-05  6e-04 0:24.6
  500   5000 -2.529411885766374e+04 5.9e+02 4.41e-04  8e-07  3e-04 0:31.1
  600   6000 -2.529411998906051e+04 1.1e+04 2.27e-04  3e-08  2e-04 0:36.3
NOTE (module=cma, iteration=612):  
condition in coordinate system exceeded 1.0e+08, rescaled to 1.0e+00, 
condition changed from 2.5e+08 to 6.3e+02
  700   7000 -2.529412006230191e+04 4.6e+01 1.81e-05  1e-10  2e-05 0:42.3
  800   8000 -2.529412006270022e+04 6.0e+02 1.79e-06  8e-13  2e-06 0:49.8
  892   8920 -2.529412006270236e+04 1.0e+04 1.04e-07  4e-15  2e-07 0:56.6
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 17:25:00 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.430236795274375e+04 1.0e+00 7.25e-01  7e-01  7e-01 0:00.1
    2     20 -2.4555000

  965   9650 -2.579291796983720e+04 6.2e+01 2.63e-06  3e-07  3e-06 0:24.6
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=1234, Wed Sep  7 17:30:43 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 -2.445770982523165e+04 1.0e+00 7.56e-01  7e-01  8e-01 0:00.0
    2     20 -2.444939588653478e+04 1.2e+00 7.18e-01  7e-01  8e-01 0:00.0
    3     30 -2.446683002767400e+04 1.3e+00 7.18e-01  7e-01  8e-01 0:00.1
  100   1000 -2.464623592640482e+04 4.0e+00 3.17e-01  2e-01  3e-01 0:01.8
  200   2000 -2.538697508384873e+04 8.3e+00 1.94e-02  4e-03  2e-02 0:04.2
  300   3000 -2.539360639948644e+04 3.3e+01 3.31e-04  1e-05  3e-04 0:06.7
  400   4000 -2.539361682954854e+04 2.8e+02 3.44e-05  2e-07  5e-05 0:09.1
  500   5000 -2.539361695839463e+04 2.9e+03 6.77e-06  6e-09  1e-05 0:11.5
NOTE (module=cma, iteration=580):  
condition in coordinate system exceeded 1.0e+08, rescaled to 1.0e+00, 
condition changed from 1.7e+08 to 1.9e+06
  600   6000 -2.5393616

In [ ]:
%%time
n_starting_points = 30
maxits= 30000
# for maximum of 5. 
for i in df_all.keys():
    cma_region(new_ss_regions_10[i], n_starting_points, maxits, df_all[i], df_10[i], best_sigma[i])

In [ ]:
%%time
n_starting_points = 30
maxits= 30000
# for maximum of 5. 
for i in df_all.keys():
    cma_region(new_ss_regions_10[i], n_starting_points, maxits, df_all[i], df_all[i], best_sigma[i])